# VacationPy
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv file
# use "keep_default_na=False" to prevent pandas from interpreting country value 'NA' as NaN
cities_df = pd.read_csv('../output_data/cities.csv', keep_default_na=False)
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,dikson,98,RU,1586555403,96,73.51,80.55,12.51,7.74
1,mahebourg,75,MU,1586554883,78,-20.41,57.70,79.00,13.87
2,pangobilian,29,PH,1586555403,76,8.80,117.86,79.65,4.47
3,kutum,0,SD,1586555404,14,14.20,24.67,71.33,9.82
4,vaini,40,TO,1586555404,69,-21.20,-175.20,78.80,8.05


In [3]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567 entries, 0 to 566
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        567 non-null    object 
 1   Cloudiness  567 non-null    int64  
 2   Country     567 non-null    object 
 3   Date        567 non-null    int64  
 4   Humidity    567 non-null    int64  
 5   Lat         567 non-null    float64
 6   Lng         567 non-null    float64
 7   Max Temp    567 non-null    float64
 8   Wind Speed  567 non-null    float64
dtypes: float64(4), int64(3), object(2)
memory usage: 40.0+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = cities_df[['Lat', 'Lng']]
display(locations.head())

# Create humidity series and convert to float
humidity = cities_df['Humidity'].astype('float')
humidity.head()

,Lat,Lng
0,73.51,80.55
1,-20.41,57.70
2,8.80,117.86
3,14.20,24.67
4,-21.20,-175.20


0    96.0
1    78.0
2    76.0
3    14.0
4    69.0
Name: Humidity, dtype: float64

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Plot heatmap
fig = gmaps.figure(center=[15, 15], zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_weather = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 70) 
                             & (cities_df['Wind Speed'] < 8) & (cities_df['Cloudiness'] == 0)]
ideal_weather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
24,marzuq,0,YE,1586555406,42,14.40,46.47,71.37,3.31
109,manokwari,0,ID,1586555537,69,-0.87,134.08,79.92,4.45
142,belsand,0,IN,1586555601,17,26.45,85.40,77.77,4.00
173,daur,0,PK,1586555604,57,26.46,68.32,77.00,4.70
181,najran,0,SA,1586555605,36,17.49,44.13,78.80,2.28
184,edeia,0,BR,1586555666,66,-17.34,-49.93,72.07,6.46
195,port hedland,0,AU,1586555667,88,-20.32,118.57,71.60,6.93
273,kanbe,0,MM,1586555798,88,16.71,96.00,77.00,6.93
292,santa fe,0,AR,1586555695,46,-31.63,-60.70,73.99,1.99
360,sangamner,0,IN,1586555868,22,19.57,74.22,79.09,3.94


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create dataframe with columns City, Country, Lat, Lng and reste index
hotel_df = ideal_weather[['City', 'Country', 'Lat', 'Lng']].reset_index(drop=True)

# add a "Hotel Name" column to dataframe
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,marzuq,YE,14.40,46.47,
1,manokwari,ID,-0.87,134.08,
2,belsand,IN,26.45,85.40,
3,daur,PK,26.46,68.32,
4,najran,SA,17.49,44.13,
5,edeia,BR,-17.34,-49.93,
6,port hedland,AU,-20.32,118.57,
7,kanbe,MM,16.71,96.00,
8,santa fe,AR,-31.63,-60.70,
9,sangamner,IN,19.57,74.22,


In [8]:
# Build URL using the Google Maps API

# Set base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters for request
params = {'type': 'lodging',
          'keyword': 'hotel',
          'radius': 5000,
          'key': g_key}

# Loop through the hotle_df and get hotel name of first hotel
for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    
    # update location key value
    params['location'] = f"{lat},{lng}"
    
    # make request
    hotels_rq = requests.get(base_url, params).json()
    
    # convert to json
    # hotels_rq = hotels_rq.json()
    
    try:
    # Update datafarame
        hotel_df.loc[index, 'Hotel Name'] = hotels_rq['results'][0]['name']
    except (KeyError, IndexError):
        print(f"Hotel in {row['City']} not found. Skipping...")
        hotel_df.drop(index, inplace=True)

# Display datafarame
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

Hotel in marzuq not found. Skipping...
Hotel in belsand not found. Skipping...
Hotel in najran not found. Skipping...
Hotel in kanbe not found. Skipping...


,City,Country,Lat,Lng,Hotel Name
0,manokwari,ID,-0.87,134.08,Swiss Belhotel Manokwari
1,daur,PK,26.46,68.32,اسٹاپ نئوآباد
2,edeia,BR,-17.34,-49.93,Hotel Jâmbore
3,port hedland,AU,-20.32,118.57,ibis Styles Port Hedland
4,santa fe,AR,-31.63,-60.70,InterTower Hotel
5,sangamner,IN,19.57,74.22,Malpani Health Club & Resort
6,cockburn town,TC,21.46,-71.14,Bohio Dive Resort


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))